<a href="https://colab.research.google.com/github/almutareb/InnovationPathfinderAI/blob/main/mixtral_react_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -U langchain_community -q
!pip install -U langchain -q
!pip install google-search-results -q
!pip install langchainhub -q
!pip install text_generation -q
!pip install arxiv -q
!pip install --upgrade --quiet  wikipedia

In [7]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')
os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

In [33]:
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

In [64]:
wikipedia.name

'wikipedia'

In [84]:
websearch = GoogleSearchAPIWrapper()

def top5_results(query):
  return websearch.results(query, 5)

google_search = Tool(
    name="google_search",
    description="Search Google for recent results.",
    #func=top5_results,
    func=websearch.run,
)

In [85]:
google_search.run("large language model")

'LLMs are artificial neural networks, the largest and most capable of which are built with a transformer-based architecture. Some recent implementations are\xa0... Large language model definition. A large language model (LLM) is a deep learning algorithm that can perform a variety of natural language processing (NLP) tasks\xa0... What are Large Language Models? Large language models (LLM) are very large deep learning models that are pre-trained on vast amounts of data. The underlying\xa0... Jan 26, 2023 ... What Are Large Language Models Used For? Large language models recognize, summarize, translate, predict and generate text and other forms of\xa0... Large language models (LLMs) are deep learning algorithms that can recognize, summarize, translate, predict, and generate content using very large datasets. Jul 20, 2023 ... A large language model is a transformer model on a large scale. It is so large that usually cannot be run on a single computer. Hence it is\xa0... Learn how the AI a

In [86]:
# HF libraries
from langchain.llms import HuggingFaceHub

# Load the model from the Hugging Face Hub
model_id = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={
    "temperature":0.1,
    "max_new_tokens":1024,
    "repetition_penalty":1.2,
    "return_full_text":False
    })

In [92]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain.tools.render import render_text_description
from langchain.tools.retriever import create_retriever_tool
from langchain.retrievers import ArxivRetriever
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)

#tools = load_tools(
#    ["arxiv"]
#)
retriever = ArxivRetriever()

arxiv_search = create_retriever_tool(
    retriever,
    "arxiv database",
    "Search arxiv database for scientific research papers and studies",
)

tools = [arxiv_search, google_search]

#prompt = hub.pull("hwchase17/react")
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
chat_model_with_stop = model_id.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
#    | model_id
    | ReActJsonSingleInputOutputParser()
)

#agent = create_react_agent(model_id, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
prompt.pretty_print()

In [93]:
agent_executor.invoke(
    {
         "input": "What's corrective retrieval augmeneted generation?"
    }
)



> Entering new AgentExecutor chain...
Thought: The term 'corrective retrieval augmented generation' seems specific to natural language processing or machine learning. However, I don't have enough information to provide a detailed explanation. I need to search for more context or related works on this topic.

Action:
```json
{
  "action": "google_search",
  "action_input": "Corrective Retrieval Augmented Generation"
}
```Jan 29, 2024 ... Although retrieval-augmented generation (RAG) is a practicable complement to LLMs, it relies heavily on the relevance of retrieved documents, ... Feb 2, 2024 ... [3] Corrective Retrieval Augmented Generation. This paper from USTC/Google Research proposes the Corrective Retrieval Augmented Generation ... Proposes Corrective Retrieval Augmented Generation (CRAG) to improve the robustness of generation in a RAG system. The core idea is to implement a self-correct ... Jan 30, 2024 ... Corrective RAG Proposes Corrective Retrieval Augmented Generation (CRAG

{'input': "What's corrective retrieval augmeneted generation?",
 'output': 'Corrective Retrieval Augmented Generation (CRAG) is a method proposed to improve the robustness of generation in a Retrieval-Augmented Generation (RAG) system. CRAG focuses on enhancing the accuracy of generated text by implementing a self-correction mechanism during the generation process, which utilizes evaluated and augmented retrieved documents.'}

In [94]:
agent_executor.invoke(
    {
         "input": "What's the latest paper on corrective retrieval augmeneted generation?"
    }
)



> Entering new AgentExecutor chain...
Thought: To find the most up-to-date information on this topic, I will first search the arXiv database for any relevant preprints related to "corrective retrieval augmented generation". If there aren't many or if they seem outdated, I will then perform a broader web search using Google Scholar to ensure more current findings.

Action:
```json
{
  "action": "arxiv database",
  "action_input": {
    "query": "corrective retrieval augmented generation",
    "max_results": 5
  }
}
```
Large language models (LLMs) inevitably exhibit hallucinations since the
accuracy of generated texts cannot be secured solely by the parametric
knowledge they encapsulate. Although retrieval-augmented generation (RAG) is a
practicable complement to LLMs, it relies heavily on the relevance of retrieved
documents, raising concerns about how the model behaves if retrieval goes
wrong. To this end, we propose the Corrective Retrieval Augmented Generation
(CRAG) to improve th

{'input': "What's the latest paper on corrective retrieval augmeneted generation?",
 'output': 'Unfortunately, due to limited information, I cannot specify the latest paper on corrective retrieval augmented generation. Three relevant papers discuss improvements in generation robustness, statistical guarantees for retrieval augmented question answering systems, and a general overview of retrieval-augmented text generation. Determining their individual publication dates and searching Google Scholar for more recent publications would aid in identifying the newest resource.'}